# Membres :

## Foromo Daniel Soromou  1759116
## Gilles Eric Zagre   1146014
## Roger Lobe 1679117

# Importe tous les modules nécessaires

### Cellule qui installe les modules necessaires.

In [1]:
!pip install --user sklearn
!pip install --user scipy
!pip install --user gensim
!pip install --user nltk
!pip install --user smart_open
!pip install --user progressbar2
!pip install --user testfixtures
!pip install --user botocore --upgrade

You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.0, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --

__IMPORTANT__: Redemarrez le notebook apres avoir execute cette cellule!

In [1]:
from __future__ import print_function

import botocore


import locale, random, glob, urllib, tarfile, sys, os, datetime, multiprocessing
import numpy as np
from time import time, clock
from pprint import pprint;
from math import floor;

from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics

import gensim
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1

from collections import OrderedDict, namedtuple

cores = multiprocessing.cpu_count()

from smart_open import smart_open
try:
    from nltk.tokenize import word_tokenize
    from nltk.stem.snowball import SnowballStemmer
except:
    import nltk
    nltk.download("punkt")
    nltk.download('stopwords')
    from nltk.tokenize import word_tokenize
    from nltk.stem.snowball import SnowballStemmer

import progressbar

# fix random seed for reproducibility
np.random.seed(1)

# Téléchargement des données

Cette cellule télécharge le dataset ImDB Sentiment de Stanford. Remplacez le root_path par le path où vous voulez télécharger les données. Un prétraitement très rapide a été effectué pour remplacer les balises <*br /> par des espaces.

In [2]:
'''
==============ÉCRIVEZ DU CODE ICI====================
'''
import re
def clean_balise(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', raw_html )
    cleantext = re.sub(' +', " ", cleantext)
    return cleantext

def replace_ponctuation(raw_text):
    raw_text = raw_text.replace("...", "&&&")
    raw_text = raw_text.replace(".", " . ").replace(",", " , ").replace(";", " ; ").replace(":", " : ").replace("!", " ! ").replace("?", " ? ").replace("&&&", " &&& ").replace("(", " ( ").replace(")", " ) ").replace('"', ' " ')
                        
    return re.sub(' +', " ", raw_text).replace("&&&", "...")

# Enlevez les balises html et séparez les ponctuations du texte
def normalize_text(text):
    # Remplacez les balises par des espaces ici
    norm_text = replace_ponctuation(clean_balise(text).strip())
    print(norm_text[:100])
    return norm_text
'''
==============FIN DE VOTRE CODE=====================
'''

'\n==============FIN DE VOTRE CODE=====================\n'

In [3]:
root_path = './'

dirname = 'aclImdb'
filename = 'aclImdb_v1.tar.gz'
dirpath = os.path.join(root_path,'imdb/')
filepath = os.path.join('./imdb/', filename)
urlpath = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

if sys.version > '3':
    control_chars = [chr(0x85)]
else:
    control_chars = [unichr(0x85)]

if not os.path.exists('./imdb/'):
    os.makedirs('./imdb/')

start = clock()

if not os.path.isfile(dirname+'/alldata-id.txt'):
    if not os.path.isdir(dirname):
        if not os.path.isfile(filepath):
            # Download IMDB archive
            print("Downloading IMDB archive...")
            urllib.request.urlretrieve(urlpath, filepath);
        print("Extracting IMDB archive...")
        tar = tarfile.open(filepath, mode='r')
        tar.extractall()
        tar.close()

    # Concatenate and normalize test/train data
    print("Cleaning up dataset...")
    folders = ['train/pos', 'train/neg', 'test/pos', 'test/neg', 'train/unsup']
    alldata = u''
    for fol in folders:
        temp = u''
        output = fol.replace('/', '-') + '.txt'
        # Is there a better pattern to use?
        txt_files = glob.glob(os.path.join(dirname, fol, '*.txt'))
        for txt in txt_files:
            with smart_open(txt, "rb") as t:
                t_clean = t.read().decode("utf-8")
                for c in control_chars:
                    t_clean = t_clean.replace(c, ' ')
                temp += t_clean
            temp += "\n"
        temp_norm = normalize_text(temp)
        
        
        with smart_open(os.path.join(dirname, output), "wb") as n:
            n.write(temp_norm.encode("utf-8"))
        alldata += temp_norm

    with smart_open(os.path.join(dirname, 'alldata-id.txt'), 'wb') as f:
        for idx, line in enumerate(alldata.splitlines()):
            num_line = u"{1}\n".format(idx, line)
            f.write(num_line.encode("utf-8"))
else:
    print('The data has already been downloaded')
    
    # Concatenate and normalize test/train data
    print("Cleaning up dataset...")
    folders = ['train/pos', 'train/neg', 'test/pos', 'test/neg', 'train/unsup']
    alldata = u''
    for fol in folders:
        temp = u''
        output = fol.replace('/', '-') + '.txt'
        # Is there a better pattern to use?
        txt_files = glob.glob(os.path.join(dirname, fol, '*.txt'))
        for txt in txt_files:
            with smart_open(txt, "rb") as t:
                t_clean = t.read().decode("utf-8")
                for c in control_chars:
                    t_clean = t_clean.replace(c, ' ')
                temp += t_clean
            temp += "\n"
        temp_norm = normalize_text(temp)
        
        
        with smart_open(os.path.join(dirname, output), "wb") as n:
            n.write(temp_norm.encode("utf-8"))
        alldata += temp_norm

    with smart_open(os.path.join(dirname, 'alldata-id.txt'), 'wb') as f:
        for idx, line in enumerate(alldata.splitlines()):
            num_line = u"{1}\n".format(idx, line)
            f.write(num_line.encode("utf-8"))
    
            
end = clock()
print ("Total running time: ", end-start)

Extracting IMDB archive...
Cleaning up dataset...
Total running time:  15.060528999999999


# Ouverture des données

In [4]:
with smart_open(os.path.join(dirname, 'alldata-id.txt'), 'rb') as alldata:
    alldata = alldata.readlines()
    X_unsup = []
    X_train = []
    X_test = []
    for line_no, line in enumerate(alldata):
        if line_no // 25000 < 1:
            X_train.append(line.decode("utf-8"))
        elif line_no // 25000 < 2:
            X_test.append(line.decode("utf-8"))   
        else:
            X_unsup.append(line.decode("utf-8"))
        
y_train = [1.0]*12500 + [0.0]*12500
y_test = [1.0]*12500 + [0.0]*12500

print('***Taille de alldata: ',len(alldata))
print('***Taille de X_unsup: ',len(X_unsup))
print('***Taille de X_train: ',len(X_train))
print('***Taille de X_test: ',len(X_test))

***Taille de alldata:  100000
***Taille de X_unsup:  50000
***Taille de X_train:  25000
***Taille de X_test:  25000


# Prétraitement

### Tokenizer

Le prétraitement des données est une étape importante, en particulier lorsque vous travaillez avec du texte. Dans la science des données, c'est très souvent la tâche qui demande le plus de temps. Comme vous l'aurez compris, le but du laboratoire est d'étudier l'influence du prétraitement sur la performance. 

La fonction ci-dessous est ce que l'on appelle un "tokenizer", son but est de créer une liste de "tokens" à partir d'une phrase (single string). Le tâche peut sembler triviale à première vue, mais l'on peut vite s'apercevoir qu'elle ne l'est pas tant (ex: "It's the student's notebook." -> ['It', 'is', 'the', 'sutdent, 'notebook', '.'] ou ["It's", "the", "student's", "notebook."] ?).

En plus du tokenizer, il est possible de rajouter un stemmer qui extrait la racine des mots (ex: "generously" -> "generous")

In [21]:
'''
Parameters:
    use_nltk_tok(bool): either to use nltk word tokenizer (True) or basic space split (False)
    use_stemmer(bool): either to use the nltk stemmer (takes the root of each word)
'''
'''
==============ÉCRIVEZ DU CODE ICI====================
'''
class custom_tokenizer():
    def __init__(self, use_nltk_tok=False, use_stemmer=False):
        self.use_nltk_tok = use_nltk_tok
        self.use_stemmer = use_stemmer
        self.stemmer = SnowballStemmer("english", ignore_stopwords=True)
        
    def __call__(self, string):
        # Utilisez le tokenizer de nltk
        if self.use_nltk_tok:
            tokens = word_tokenize(string)
        # Séparez simplement les mots par les espaces
        else:
            tokens = string.split(' ')
        # 
        if self.use_stemmer:
            words = [self.stemmer.stem(token) for token in tokens] 
        else:
            words = tokens
        return words
'''
==============FIN DE VOTRE CODE=====================
'''

'\n==============FIN DE VOTRE CODE=====================\n'

### Feature Extraction

Le texte est ce que l'on appelle une donnée non structurée, c'est à dire qu'il ne possède pas d'attribut ("feature") fixe et mesurable. Or pour la majorité des algorithmes d'appentissage il est nécessaire d'avoir des vecteurs de même dimension en entrée. Ci-dessous se trouve donc l'extracteur de features, qui transforme toutes les phrases d'un corpus en vecteurs de même dimension. 

L'extraction de feature pour le texte est encore un sujet de recherche, mais quelques méthodes existantes sont déjà relativement efficaces. Les méthodes étudiées durant ce laboratoire sont l'approche Bag-of-Word et Doc2Vec.

__QUESTION 1__:
Faites une petite recherche de littérature sur ces méthodes et expliquez-les avec vos mots (5-10 lignes par méthode). Une bonne piste de départ sont les articles https://arxiv.org/pdf/1301.3781.pdf et https://machinelearningmastery.com/gentle-introduction-bag-words-model/

# Voir rapport PDF

__QUESTION 2__: D'après vous, quels sont les avantages de Doc2Vec face à BoW?

# Voir rapport PDF

__QUESTION 3__: Décrivez rapidement qu'est qu'un bigram. En quoi peut-il améliorer les performances?

# Voir rapport PDF

### Extracteur

In [25]:
'''
Parameters:
    X_unsup(list): list of sentences (str) with no label
    X_train(list): list of sentences (str) of the training set
    X_test(list): list of sentences (str) of the test set
    use_Doc2Vec(bool): either to use Doc2Vec (True) or Bag-of-words (False)
    use_bigram(bool): either to create bigrams with the most common sequence of two words (ex: ['New', 'York'] -> ['New_York'])
    use_nltk_tok(bool): either to use nltk word tokenizer (True) or basic space split (False)
    use_stemmer(bool): either to use the nltk stemmer (takes the root of each word)
'''

def prepare_data(X_unsup, X_train, X_test, use_Doc2Vec=False, use_bigram=False, use_nltk_tok=False, use_stemmer=False):
    
    X =  X_train + X_test + X_unsup
    
    kwargs = {"use_nltk_tok":use_nltk_tok, "use_stemmer":use_stemmer}
    tokenizer = custom_tokenizer(**kwargs)
    
    if use_Doc2Vec:
        
        SentimentDocument = namedtuple('SentimentDocument', 'words tags')
        
        print('Creating the docs for Doc2Vec...')
        allreviews = []
        for review in X:
            words = tokenizer(review)
            allreviews.append(words)
        
        if use_bigram:
            print('Training bigram transormer...')
            phrases = Phrases(allreviews)
            bigram = Phraser(phrases)
            # Apellez bigram pour chaque review de la liste allreviews. 
            # Il va permettre de remplacer les couples de mots qui reviennent souvent par un seul token
            allreviews = [bigram[review] for review in allreviews]
        
        alldocs = []
        for line_no, words in enumerate(allreviews):
            tags = [line_no] 
            alldocs.append(SentimentDocument(words, tags))
        
        # Ici il s'agît de récupérer les documents qui serviront pour le test
        test_docs = alldocs[len(X_train):-len(X_unsup)]
        print('*** Taille de testdocs: ',len(test_docs))
        # Ici il faut récupérer les documents qui serviront pour l'entraîenement du model
        train_docs = alldocs[:len(X_train)] + alldocs[-len(X_unsup):]
        
        print('Creating Doc2Vec models...')
        simple_models = [
            # PV-DBOW 
            Doc2Vec(dm=0, vector_size=vec_size, negative=5, hs=0, min_count=2, workers=cores),
            # PV-DM w/ average
            Doc2Vec(dm=1, dm_mean=1, vector_size=vec_size, window=10, negative=5, hs=0, min_count=2, workers=cores),
        ]

        print('Creating a vocabulary...')
        simple_models[0].build_vocab(train_docs)
        for model in simple_models[1:]:
            model.reset_from(simple_models[0])

        models_by_name = OrderedDict((str(model), model) for model in simple_models)
        models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])

        alpha, min_alpha, passes = (0.025, 0.001, 20) 
        alpha_delta = (alpha - min_alpha) / passes
        print('Vectorizing the data... %s' % datetime.datetime.now())
        
        print('Starting training...')
        for epoch in range(passes):
            random.shuffle(train_docs)  # Shuffling gets best results

            for name, train_model in models_by_name.items():
                # Train
                train_model.alpha, train_model.min_alpha = alpha, alpha
                train_model.train(train_docs, total_examples=len(train_docs), epochs=1)

            print('Completed pass %i at alpha %f' % (epoch + 1, alpha))
            alpha -= alpha_delta

        print('Vectorization done %s' % str(datetime.datetime.now()))
        
        X_train_vect = np.zeros((len(X_train), vec_size*2))
        X_test_vect = np.zeros((len(X_test), vec_size*2))
        
        # Maintenant remplacez les reviews par leur vecteurs 
        # !!ATTENTION!! Dans le diminuer le biais dans l'expérience, 
        # il faut inferrer les vecteurs pour l'ensemble de test
        print('Inferring vectors...')
        for i in range(len(X_train)):
            # Simplement le .docvec du document
            X_train_vect[i,:] = models_by_name['dbow+dmm'].docvecs[i]
            # ici il faut infer_vector depuis le modèle entraîné précédemment
            X_test_vect[i,:] = models_by_name['dbow+dmm'].infer_vector(alldocs[i].words)
    
    else:
        if use_bigram:
            ngram_range = (1,2)
        else:
            ngram_range = (1,1)

        print('Creating a vocabulary...')

        CV = CountVectorizer(analyzer="word", tokenizer=tokenizer, ngram_range=ngram_range, max_features=vocab_size)
        
        CV.fit_transform(X_train + X_unsup)

        vocab = CV.vocabulary_
        stop_words = CV.stop_words_

        print('Vectorizing the data... %s' % datetime.datetime.now())

        CV = CountVectorizer(analyzer="word", ngram_range=ngram_range, tokenizer=custom_tokenizer(**kwargs), preprocessor=lambda text: text.replace("<br />", " "), vocabulary=vocab, stop_words=stop_words)

        X_train_vect = CV.fit_transform(X_train)
        X_test_vect = CV.fit_transform(X_test)

        print('Vectorization done %s' % str(datetime.datetime.now()))
        
    return X_train_vect, X_test_vect

Ci-dessous, se trouve la ligne à executer pour obtenir les vecteurs à partir de chaque document du corpus. Changez kwargs pour avoir différents prétraitements et feature extracteur.

NB: Certains algorithmes sont plus long que d'autres (si tout est activé), soyez patients.

In [16]:
import nltk
nltk.download('stopwords')
kwargs = {
    "use_Doc2Vec":True, 
    "use_bigram":False, 
    "use_nltk_tok":False, 
    "use_stemmer":False
}

vocab_size = None # For the BoW approach. No bound when set to None
vec_size = 10 # For the Doc2Vec approach.

X_train_vec, X_test_vec = prepare_data(X_unsup, X_train, X_test, **kwargs)

n_features = X_train_vec.shape[1]

print('Number of features', n_features)
print('Number of features', X_test_vec.shape[1])

[nltk_data] Downloading package stopwords to
[nltk_data]     /usagers/gizag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Creating the docs for Doc2Vec...
*** Taille de testdocs:  25000
Creating Doc2Vec models...
Creating a vocabulary...
Vectorizing the data... 2018-05-21 15:53:00.270993
Starting training...
Completed pass 1 at alpha 0.025000
Completed pass 2 at alpha 0.023800
Completed pass 3 at alpha 0.022600
Completed pass 4 at alpha 0.021400
Completed pass 5 at alpha 0.020200
Completed pass 6 at alpha 0.019000
Completed pass 7 at alpha 0.017800
Completed pass 8 at alpha 0.016600
Completed pass 9 at alpha 0.015400
Completed pass 10 at alpha 0.014200
Completed pass 11 at alpha 0.013000
Completed pass 12 at alpha 0.011800
Completed pass 13 at alpha 0.010600
Completed pass 14 at alpha 0.009400
Completed pass 15 at alpha 0.008200
Completed pass 16 at alpha 0.007000
Completed pass 17 at alpha 0.005800
Completed pass 18 at alpha 0.004600
Completed pass 19 at alpha 0.

## Classifieur

Maintenant que les features sont extraits, il faut entraîner un classifieur. Une regression logistique a été choisi pour cette tâche, mais si vous êtes curieux, vous pouvez en essayer d'autres (Les classifieurs simples les plus populaires pour la classification de texte sont: Bayes Naif, SVM linéaire mais vous pouvez choisir autre chose).

N.B.: Ici, un Grid Search avec Cross-Validation a été mit en place pour vous familiariser avec les méthodes d'ensemble qui seront le sujet du laboratoire 2. Il n'est pas nécessaire de le comprendre pour le moment, mais si vous êtes curieux, vous pouvez vous renseigner sur le sujet.

In [26]:
def Classifier(X_train, y_train, X_test, y_test):

    clf = Pipeline([
        ('clf', LogisticRegression())
    ]);

    '''
        Parameters
        ----------
        penalty : string, 'l1' or 'l2' (default='l2')
            Used to specify the norm used in the penalization. The 'newton-cg',
            'sag' and 'lbfgs' solvers support only l2 penalties.
        dual : bool, (default=True)
            Dual or primal formulation. Dual formulation is only implemented for
            l2 penalty with liblinear solver. Prefer dual=False when
            n_samples > n_features.
        tol : float, optional (default=1e-4)
            Tolerance for stopping criteria.
        C : float, optional (default=1.0)
            Inverse of regularization strength; must be a positive float.
            Like in support vector machines, smaller values specify stronger
            regularization.
        fit_intercept : boolean, optional (default=True)
            Specifies if a constant (a.k.a. bias or intercept) should be
            added to the decision function.
        max_iter : int, (default=1000)
            The maximum number of iterations to be run.
        solver : string, {'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'}, (default: 'liblinear')
            Algorithm to use in the optimization problem.
            - For small datasets, 'liblinear' is a good choice, whereas 'sag' and
                'saga' are faster for large ones.
            - For multiclass problems, only 'newton-cg', 'sag', 'saga' and 'lbfgs'
                handle multinomial loss; 'liblinear' is limited to one-versus-rest
                schemes.
            - 'newton-cg', 'lbfgs' and 'sag' only handle L2 penalty, whereas
                'liblinear' and 'saga' handle L1 penalty.
            Note that 'sag' and 'saga' fast convergence is only guaranteed on
            features with approximately the same scale. You can
            preprocess the data with a scaler from sklearn.preprocessing.
    '''

    parameters = {
#         'clf__penalty': ('l2', 'l1'),
#         'clf__dual':(True, False),
#         'clf__tol':(1e-2, ),
#         'clf__C': (0.1, ),
#         'clf__fit_intercept': (True, False).
        'clf__max_iter':(300,),
#         'solver':('liblinear',)
    }

    grid_search = GridSearchCV(clf, parameters,  verbose=1, iid=True, cv=4, n_jobs=-1, return_train_score=False)

    print("Performing grid search...")
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(X_train_vec, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()
    best_estimator = grid_search.best_estimator_
    score_train = grid_search.best_score_
    print("Best score for validation set : %0.3f" % score_train)
    
    tic= time()
    predictions = best_estimator.predict(X_test_vec);
    print("Prediction done in %0.3fs" % (time() - tic))
    score_test = metrics.accuracy_score(y_test, predictions)
    print("Best score for test set : %0.3f" % score_test)
    print(metrics.classification_report(y_test, predictions, digits=4))
    
    return score_train, score_test

In [18]:
train_score, test_score = Classifier(X_train_vec, y_train, X_test_vec, y_test)

Performing grid search...
parameters:
{'clf__max_iter': (300,)}
Fitting 4 folds for each of 1 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.3s finished


done in 1.180s

Best score for validation set : 0.831
Prediction done in 0.001s
Best score for test set : 0.501
             precision    recall  f1-score   support

        0.0     0.5098    0.0519    0.0942     12500
        1.0     0.5005    0.9501    0.6556     12500

avg / total     0.5052    0.5010    0.3749     25000



__QUESTION 4__: Pour l'approche BoW, tracez sur un même graphique les précisions obtenues sur l'ensemble d'entraînement et de test, pour chaque option de prétraitement, en faisant varier la taille du vocabulaire (au moins :1000, 10000, 100000, None). En résumé, un total de 6 courbes avec la précision en y, et la taille du vocabulaire en x). __Commentez les résultats__.

# Voir rapport PDF

In [ ]:
# Option BoW:

# Construction de la liste des options d'execution
lok = []
lok.append({"use_Doc2Vec":False, "use_bigram":False, "use_nltk_tok":False, "use_stemmer":False })
lok.append({"use_Doc2Vec":False, "use_bigram":False, "use_nltk_tok":False, "use_stemmer":True })
lok.append({"use_Doc2Vec":False, "use_bigram":False, "use_nltk_tok":True, "use_stemmer":False })
lok.append({"use_Doc2Vec":False, "use_bigram":False, "use_nltk_tok":True, "use_stemmer":True })
lok.append({"use_Doc2Vec":False, "use_bigram":True, "use_nltk_tok":False, "use_stemmer":False })
lok.append({"use_Doc2Vec":False, "use_bigram":True, "use_nltk_tok":False,  "use_stemmer":True})
lok.append({"use_Doc2Vec":False, "use_bigram":True, "use_nltk_tok":True, "use_stemmer":False })
lok.append({"use_Doc2Vec":False, "use_bigram":True, "use_nltk_tok":True, "use_stemmer":True })

# Pour chaque set de configuration (options)
for args in lok:
    list_of_train_score = []
    list_of_test_score = []
    n_feature = -1
    
    # Pour differentes taille de vocabulaire (None correspond a non specifie)
    for k in [1000, 10000, 100000, None]:
        vocab_size = k
        
        # Pretraitement des donnees et extraction de features
        X_train_vec, X_test_vec = prepare_data(X_unsup, X_train, X_test, **args)
        if k == None : 
            n_feature = X_train_vec.shape[1]
            
        # Classification
        train_score, test_score = Classifier(X_train_vec, y_train, X_test_vec, y_test)
        
        # Memorisation des donnees
        list_of_train_score.append(train_score)
        list_of_test_score.append(test_score)

__QUESTION 5__: Pour l'approche Doc2Vec, tracez sur un même graphique les précisions obtenues sur l'ensemble d'entraînement et de test, pour chaque option de prétraitement, en faisant varier le nombre de features (vec_size) (au moins :4, 8, 64, 256). En résumé, un total de 6 courbes avec la précision en y, et la taille du vecteur en x). __Commentez les résultats__.

# Voir rapport PDF

In [27]:
# Option Doc2Vec:

import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Liste pour faire varier le nombre de features (vec_size): 
vec_size_list = [4, 8, 64, 256]

# Liste pour faire varier les options de pretraitement
kwargsDoc2Vec_1 = {"use_Doc2Vec":True, "use_bigram":False, "use_nltk_tok":False, "use_stemmer":False}
kwargsDoc2Vec_2 = {"use_Doc2Vec":True, "use_bigram":False, "use_nltk_tok":False, "use_stemmer":True}
kwargsDoc2Vec_3 = {"use_Doc2Vec":True, "use_bigram":False, "use_nltk_tok":True, "use_stemmer":False}
kwargsDoc2Vec_4 = {"use_Doc2Vec":True, "use_bigram":False, "use_nltk_tok":True, "use_stemmer":True}
kwargsDoc2Vec_5 = {"use_Doc2Vec":True, "use_bigram":True, "use_nltk_tok":False, "use_stemmer":False}
kwargsDoc2Vec_6 = {"use_Doc2Vec":True, "use_bigram":True, "use_nltk_tok":False, "use_stemmer":True}
kwargsDoc2Vec_7 = {"use_Doc2Vec":True, "use_bigram":True, "use_nltk_tok":True, "use_stemmer":False}
kwargsDoc2Vec_8 = {"use_Doc2Vec":True, "use_bigram":True, "use_nltk_tok":True, "use_stemmer":True}

kwargs_List = [kwargsDoc2Vec_1, kwargsDoc2Vec_2, kwargsDoc2Vec_3, kwargsDoc2Vec_4, kwargsDoc2Vec_5, kwargsDoc2Vec_6, kwargsDoc2Vec_7, kwargsDoc2Vec_8]

# Pour chaque option de pretraitement
RESULTS_TAB = []
i=1
for kwargs in kwargs_List:
    # Pour chaque taille de vecteur
    print('***OPTION : ',i)
    i+=1
    for vec_size in vec_size_list: 
        print('   ***VecSize =',vec_size)
        
        Resultats=[]
        
        # Pretraitement des donnees et extraction des features
        X_train_vec, X_test_vec = prepare_data(X_unsup, X_train, X_test, **kwargs)
        n_features = X_train_vec.shape[1]
        print('Number of features', n_features)
        
        # Classification
        train_score, test_score = Classifier(X_train_vec, y_train, X_test_vec, y_test)
        
        # Rajouter les resultats pour cette taille de vecteur
        Resultats.append([vec_size, train_score, test_score])
        print('      *** Results : ', i-1,', ',vec_size,', ',train_score,', ' ,test_score)
    
    # Rajouter les Resultats pour ce set d'option    
    RESULTS_TAB.append(Resultats)
        


[nltk_data] Downloading package stopwords to
[nltk_data]     /usagers/gizag/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usagers/gizag/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
***OPTION :  1
   ***VecSize = 4
Creating the docs for Doc2Vec...
*** Taille de testdocs:  25000
Creating Doc2Vec models...
Creating a vocabulary...
Vectorizing the data... 2018-05-21 21:53:35.369903
Starting training...
Completed pass 1 at alpha 0.025000
Completed pass 2 at alpha 0.023800
Completed pass 3 at alpha 0.022600
Completed pass 4 at alpha 0.021400
Completed pass 5 at alpha 0.020200
Completed pass 6 at alpha 0.019000
Completed pass 7 at alpha 0.017800
Completed pass 8 at alpha 0.016600
Completed pass 9 at alpha 0.015400
Completed pass 10 at alpha 0.014200
Completed pass 11 at alpha 0.013000
Completed pass 12 at alpha 0.011800
Completed pass 13 at alpha 0.010600
Completed pass 14 at alpha 0.009400
Completed pass 15 at

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.3s finished


done in 1.497s

Best score for validation set : 0.691
Prediction done in 0.010s
Best score for test set : 0.684
             precision    recall  f1-score   support

        0.0     0.6583    0.7648    0.7076     12500
        1.0     0.7194    0.6030    0.6561     12500

avg / total     0.6889    0.6839    0.6818     25000

      *** Results :  1 ,  4 ,  0.69148 ,  0.68392
   ***VecSize = 8
Creating the docs for Doc2Vec...
*** Taille de testdocs:  25000
Creating Doc2Vec models...
Creating a vocabulary...
Vectorizing the data... 2018-05-21 22:02:19.093887
Starting training...
Completed pass 1 at alpha 0.025000
Completed pass 2 at alpha 0.023800
Completed pass 3 at alpha 0.022600
Completed pass 4 at alpha 0.021400
Completed pass 5 at alpha 0.020200
Completed pass 6 at alpha 0.019000
Completed pass 7 at alpha 0.017800
Completed pass 8 at alpha 0.016600
Completed pass 9 at alpha 0.015400
Completed pass 10 at alpha 0.014200
Completed pass 11 at alpha 0.013000
Completed pass 12 at alpha 0.0

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.4s finished


done in 1.580s

Best score for validation set : 0.809
Prediction done in 0.013s
Best score for test set : 0.814
             precision    recall  f1-score   support

        0.0     0.7970    0.8418    0.8188     12500
        1.0     0.8324    0.7856    0.8083     12500

avg / total     0.8147    0.8137    0.8136     25000

      *** Results :  1 ,  8 ,  0.8092 ,  0.81372
   ***VecSize = 64
Creating the docs for Doc2Vec...
*** Taille de testdocs:  25000
Creating Doc2Vec models...
Creating a vocabulary...
Vectorizing the data... 2018-05-21 22:11:55.417623
Starting training...
Completed pass 1 at alpha 0.025000
Completed pass 2 at alpha 0.023800
Completed pass 3 at alpha 0.022600
Completed pass 4 at alpha 0.021400
Completed pass 5 at alpha 0.020200
Completed pass 6 at alpha 0.019000
Completed pass 7 at alpha 0.017800
Completed pass 8 at alpha 0.016600
Completed pass 9 at alpha 0.015400
Completed pass 10 at alpha 0.014200
Completed pass 11 at alpha 0.013000
Completed pass 12 at alpha 0.0

...

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.4s finished


done in 1.510s

Best score for validation set : 0.717
Prediction done in 0.002s
Best score for test set : 0.706
             precision    recall  f1-score   support

        0.0     0.6880    0.7538    0.7194     12500
        1.0     0.7278    0.6582    0.6913     12500

avg / total     0.7079    0.7060    0.7053     25000

      *** Results :  3 ,  4 ,  0.71692 ,  0.706
   ***VecSize = 8
Creating the docs for Doc2Vec...
*** Taille de testdocs:  25000
Creating Doc2Vec models...
Creating a vocabulary...
Vectorizing the data... 2018-05-21 23:38:32.766700
Starting training...
Completed pass 1 at alpha 0.025000
Completed pass 2 at alpha 0.023800
Completed pass 3 at alpha 0.022600
Completed pass 4 at alpha 0.021400
Completed pass 5 at alpha 0.020200
Completed pass 6 at alpha 0.019000
Completed pass 7 at alpha 0.017800
Completed pass 8 at alpha 0.016600
Completed pass 9 at alpha 0.015400
Completed pass 10 at alpha 0.014200
Completed pass 11 at alpha 0.013000
Completed pass 12 at alpha 0.011

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.4s finished


done in 1.584s

Best score for validation set : 0.789
Prediction done in 0.007s
Best score for test set : 0.779
             precision    recall  f1-score   support

        0.0     0.7612    0.8138    0.7867     12500
        1.0     0.8000    0.7447    0.7714     12500

avg / total     0.7806    0.7793    0.7790     25000

      *** Results :  3 ,  8 ,  0.78904 ,  0.77928
   ***VecSize = 64
Creating the docs for Doc2Vec...


KeyboardInterrupt: 

__BONUS__: Vous pouvez aussi faire varier les autres paramètres du Doc2Vec (dm, negative, hs, min_count, window) et étudier leurs impacts sur les résultats à l'aide de courbes